In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
%matplotlib inline

In [2]:
path = 'resources/extended_dataset.parquet'
df = pd.read_parquet(path)
df.head(2)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,...,Re-inspection,Community,Neighborhood,Ward,Precinct,Violation IDs,Nb Violations,Nb Violations of high severity,Nb Violations of medium severity,Nb Violations of low severity
0,2320519,SALAM RESTAURANT,SALAM RESTAURANT,2002822,Restaurant,High,4634-4636 N KEDZIE AVE,60625,2019-10-25,Complaint,...,True,ALBANY PARK,"NORTH PARK,ALBANY PARK",33,27,[None],0,0,0,0
1,2320509,TAQUERIA EL DORADO,TAQUERIA EL DORADO,2694960,Restaurant,High,2114 W LAWRENCE AVE,60625,2019-10-25,License,...,True,LINCOLN SQUARE,LINCOLN SQUARE,47,38,"[3, 5, 25, 50, 53, 53, 56, 58]",8,2,1,5


In [7]:
np.sort(df['Community'].unique())

array(['ALBANY PARK', 'ARCHER HEIGHTS', 'ARMOUR SQUARE', 'ASHBURN',
       'AUBURN GRESHAM', 'AUSTIN', 'AVALON PARK', 'AVONDALE',
       'BELMONT CRAGIN', 'BEVERLY', 'BRIDGEPORT', 'BRIGHTON PARK',
       'BURNSIDE', 'CALUMET HEIGHTS', 'CHATHAM', 'CHICAGO LAWN',
       'CLEARING', 'DOUGLAS', 'DUNNING', 'EAST GARFIELD PARK',
       'EAST SIDE', 'EDGEWATER', 'EDISON PARK', 'ENGLEWOOD',
       'FOREST GLEN', 'FULLER PARK', 'GAGE PARK', 'GARFIELD RIDGE',
       'GRAND BOULEVARD', 'GREATER GRAND CROSSING', 'HEGEWISCH',
       'HERMOSA', 'HUMBOLDT PARK', 'HYDE PARK', 'IRVING PARK',
       'JEFFERSON PARK', 'KENWOOD', 'LAKE VIEW', 'LINCOLN PARK',
       'LINCOLN SQUARE', 'LOGAN SQUARE', 'LOOP', 'LOWER WEST SIDE',
       'MCKINLEY PARK', 'MONTCLARE', 'MORGAN PARK', 'MOUNT GREENWOOD',
       'NEAR NORTH SIDE', 'NEAR SOUTH SIDE', 'NEAR WEST SIDE', 'NEW CITY',
       'NORTH CENTER', 'NORTH LAWNDALE', 'NORTH PARK', 'NORWOOD PARK',
       'OAKLAND', 'OHARE', 'PORTAGE PARK', 'PULLMAN', 'RIVERDALE',
 

In [10]:
community_areas = pd.read_csv('resources/community_areas_infos.csv', delimiter=',').set_index('COMMUNITY AREA NAME')
community_areas.head()

,Community Area Number,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
COMMUNITY AREA NAME,,,,,,,,
Rogers Park,1.0,7.7,23.6,8.7,18.2,27.5,23939,39.0
West Ridge,2.0,7.8,17.2,8.8,20.8,38.5,23040,46.0
Uptown,3.0,3.8,24.0,8.9,11.8,22.2,35787,20.0
Lincoln Square,4.0,3.4,10.9,8.2,13.4,25.5,37524,17.0
North Center,5.0,0.3,7.5,5.2,4.5,26.2,57123,6.0
